In [2]:
#%matplotlib notebook
%matplotlib inline
import sys
sys.path.append("../../NephNet")
import matplotlib.pyplot as plt
import os
import random
import shutil
import time
import warnings
import sys
import numpy as np
import json
import argparse
import torch
from torch.optim import lr_scheduler
from tqdm import tqdm
from data_loader import data_loaders as module_data
from model import loss as module_loss
from model import metric as module_metric
from model import model as module_arch
from data_loader import databases as module_datasets
from base.base_data_loader import BaseDataLoader
from trainer import Trainer
from utils import Logger
from utils import util
from utils import tSNE_test as tSNE#import Identity, tSNE_generator
from utils import torchsummary
from utils import viewTraining
from utils import lr_finder
from utils import classActivationMap
import importlib
import math
import torchvision
from torch.nn import functional as F
from torch import topk
import skimage.transform
import jupyter
from IPython import display
from ipywidgets import *
from utils import hyperband as HypOpt
import argparse
from sklearn.model_selection import StratifiedKFold
import copy
from utils import transforms3d as t3d
import pandas as pd

import cleanlab
from cleanlab import baseline_methods
from sklearn.metrics import accuracy_score

print("Modules loaded")


importlib.reload(module_data) #load recent changes to data_loaders.py
importlib.reload(tSNE)
importlib.reload(module_arch)
importlib.reload(module_loss)
importlib.reload(module_metric)
importlib.reload(util)
importlib.reload(viewTraining)
importlib.reload(lr_finder)
importlib.reload(classActivationMap)
importlib.reload(HypOpt)
print("Reload complete")

print("GPUs available: " + str(torch.cuda.device_count()))
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

Modules loaded
Reload complete
GPUs available: 1


In [3]:
config_file = '../configs/config_pruned.json'

# load config file
with open(config_file) as handle:
    config = json.load(handle)
# setting path to save trained models and log files
path = os.path.join(config['trainer']['save_dir'], config['name'])

In [7]:
filename = config['data_loader']['args']['hdf5_path']
path = filename.split('.')
path = path[0] + "_combined.h5"
print("NEW FILE " + path)
store = pd.HDFStore(path)
with pd.HDFStore(filename) as f:
        print()
        print("Successfully created " + path)
        print("===============")
        keys = list(f.keys())
        print("{0} keys in this file: {1}".format(len(keys), keys))
        
        num_images_train = f['Metadata']['TrainingNum'][0]
        num_images_test = f['Metadata']['TestingNum'][0]
        mean = f['Metadata']['TrainingMean'][0]
        std = f['Metadata']['TrainingStd'][0]
        '''
        num_images_train = len(f['train_data'])
        num_images_test = len(f['test_data'])
        mean = np.mean(f['train_data'].to_numpy())
        std = np.std(f['train_data'].to_numpy())
        '''
        mean_t = np.mean(f['test_data'].to_numpy())
        std_t = np.std(f['test_data'].to_numpy())
        
        print("Training images: " + str(num_images_train))
        print("Testing images: " + str(num_images_test))
        print("Training image mean = " + str(mean) + " and std = " + str(std))
        print("Testing image mean = " + str(mean_t) + " and std = " + str(std_t))
        
        train_data = f['train_data'].to_numpy()
        train_label = f['train_labels'].to_numpy()
        test_data = f['test_data'].to_numpy()
        test_label = f['test_labels'].to_numpy()
        
        train_label = train_label.squeeze()
        test_label = test_label.squeeze()
        
        print(train_data.shape)
        print(test_data.shape)
        print(train_label.shape)
        print(test_label.shape)
        combined_data = np.concatenate((train_data, test_data), axis = 0)
        combined_label = np.concatenate((train_label, test_label), axis = 0)
        
                
        print(combined_data.shape)
        print(combined_label.shape)
        
        
        store.append('train_data', pd.DataFrame(combined_data.astype(np.int64)))
        store.append('train_labels', pd.DataFrame(combined_label.astype(np.int64)))
        
        
        
        
        metadata = pd.DataFrame({'TrainingNum': len(combined_data), 'TestingNum': 0, \
                'TrainingMean': np.mean(combined_data), 'TrainingStd': np.std(combined_data)}, index=[0])      
        store.append('Metadata', metadata)
        store.append('test_ids', f['test_ids'])
        print(metadata)
        
        del train_data, test_data
        print("FIN")
store.close() 

with pd.HDFStore(path) as f:
        print('\n\n\n')
        print("Successfully created " + path)
        print("===============")
        keys = list(f.keys())
        print("{0} keys in this file: {1}".format(len(keys), keys))

NEW FILE C:/Users/awoloshu/Desktop/datasets/f33f44combined_noCNT/3d_all/pruned_train_test/f33f44combined_noCNT_3d_all_prune_bad_labels1_combined.h5

Successfully created C:/Users/awoloshu/Desktop/datasets/f33f44combined_noCNT/3d_all/pruned_train_test/f33f44combined_noCNT_3d_all_prune_bad_labels1_combined.h5
6 keys in this file: ['/Metadata', '/test_data', '/test_ids', '/test_labels', '/train_data', '/train_labels']
Training images: 85375
Testing images: 13619
Training image mean = 15.747216671459945 and std = 19.44407238183732
Testing image mean = 15.739041926556125 and std = 19.47083198216387
(85375, 7168)
(13619, 7168)
(85375,)
(13619,)
(98994, 7168)
(98994,)


C:\Users\awoloshu\Documents\pytorch36\lib\site-packages\tables\leaf.py:353: RuntimeWarning: overflow encountered in long_scalars
  expected_mb = (expectedrows * rowsize) // MB


   TrainingNum  TestingNum  TrainingMean  TrainingStd
0        98994           0     15.746092    19.447756
FIN




Successfully created C:/Users/awoloshu/Desktop/datasets/f33f44combined_noCNT/3d_all/pruned_train_test/f33f44combined_noCNT_3d_all_prune_bad_labels1_combined.h5
4 keys in this file: ['/Metadata', '/test_ids', '/train_data', '/train_labels']
